https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/README_sdxl.md

In [ ]:
%%bash

git clone https://github.com/huggingface/diffusers
cd diffusers
pip install -e .

In [ ]:
%pip install -r requirements_sdxl.txt

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
%%bash
aws s3 sync s3://sagemaker-sdxl/input/dog ./dog

In [ ]:
# from huggingface_hub import snapshot_download

# local_dir = "./dog"
# snapshot_download(
#     "diffusers/dog-example",
#     local_dir=local_dir, repo_type="dataset",
#     ignore_patterns=".gitattributes",
# )


In [ ]:
%%bash

export MODEL_NAME="stabilityai/stable-diffusion-xl-base-1.0"
export INSTANCE_DIR="dog"
export OUTPUT_DIR="lora-trained-xl"
export VAE_PATH="madebyollin/sdxl-vae-fp16-fix"

accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --pretrained_vae_model_name_or_path=$VAE_PATH \
  --output_dir=$OUTPUT_DIR \
  --enable_xformers_memory_efficient_attention \
  --gradient_checkpointing \
  --use_8bit_adam \
  --mixed_precision="fp16" \
  --instance_prompt="a photo of sks dog" \
  --resolution=1024 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --seed="0"


In [ ]:
%%bash
aws s3 sync ./lora-trained-xl s3://sagemaker-sdxl/output/`date +%s`/lora-trained-xl